$s$

In [12]:
import ipywidgets as widgets
out = widgets.Output()
with out:
    
    %bookmark 'model_dir'
    current_dir = %pwd
    while 'models' in current_dir:
        %cd ..
        current_dir = %pwd
    %bookmark 'base_dir'
    %cd source_code
    out.clear_output()
%run base
%run gui_v6
with out:
    %cd ..
    %cd models/fourbar
    out.clear_output()


In [13]:
m=model()
m.show()

## VEHICLE DYNAMICS MODELING AND SIMULATION TOOL

In [14]:
m.topology.edges

OutMultiEdgeView([(rbs_ground, rbs_link, 'jcs_rev')])

In [15]:
def equations_assembler(topology):
    
    edgelist = topology.edges(data='obj')
    nodelist = topology.nodes
    
    n_nodes = len(nodelist)
    n_edges = len(edgelist)
            
    jacobian = np.zeros((n_edges+n_nodes,n_nodes),dtype=np.object)
    jacobian.fill(None)
    
    equations = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    vel_rhs = np.zeros((n_edges+n_nodes),dtype=np.object)
    acc_rhs = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    reaction_vector = np.zeros((n_edges),dtype=np.object)
    reaction_indecies = np.zeros((n_edges),dtype=np.object)
    
    lamda_vector = np.zeros((n_edges+n_nodes),dtype=np.object)
    
    node_index = dict( (node,i) for i,node in enumerate(nodelist) )
    
    n7 = np.arange(7)
    for ei,e in enumerate(edgelist):
        (u,v) = e[:2]
        eo    = e[2]
        
        ui = node_index[u]
        vi = node_index[v]
        
        ui7 = (ui*7)+n7
        vi7 = (vi*7)+n7
        
        if jacobian[ui+n_edges,ui]==None: jacobian[ui+n_edges,ui] = (u.jac,ui7)
        if jacobian[vi+n_edges,vi]==None: jacobian[vi+n_edges,vi] = (v.jac,vi7)
                
        jacobian[ei,ui] = (eo.jacobian_i,ui7,vi7)
        jacobian[ei,vi] = (eo.jacobian_j,ui7,vi7)
        
        equations[ei] = (eo.equations,ui7,vi7)
        if equations[ui+n_edges]==0: equations[ui+n_edges]=(u.equations,ui7)
        if equations[vi+n_edges]==0: equations[vi+n_edges]=(v.equations,vi7)
        
        
        vel_rhs[ei] = ((eo.vel_rhs) if eo.nc==1 else np.zeros((eo.nc,1)))
        vel_rhs[ui+n_edges] = (np.array([[0]]) if u.nc==1 else np.zeros((7,1)))
        vel_rhs[vi+n_edges] = (np.array([[0]]) if v.nc==1 else np.zeros((7,1)))
        
        acc_rhs[ei] = (eo.acc_rhs,ui7,vi7)
        if acc_rhs[ui+n_edges]==0: acc_rhs[ui+n_edges]=(u.acc_rhs,ui7)
        if acc_rhs[vi+n_edges]==0: acc_rhs[vi+n_edges]=(v.acc_rhs,vi7)
            
        reaction_vector[ei] = (eo.reactions,ui7,vi7,eo.index)
        reaction_indecies[ei] = eo.reaction_index
        
        lamda_vector[ei] = eo.index
        if lamda_vector[ui+n_edges]==0: lamda_vector[ui+n_edges]=u.index
        if lamda_vector[vi+n_edges]==0: lamda_vector[vi+n_edges]=v.index
        
    #####################################################
    # Defining vectorized functions
    #####################################################
    
    def j_mapper(i,q):
        if len(i)==3:
            fun,bi,bj = i
            return fun(q[bi],q[bj])
        else:
            fun,bi = i
            return fun(q[bi])
    j_vectorized = np.vectorize(j_mapper,otypes=[np.object],excluded='q')
    
    
    def v_mapper(i):
        if isinstance(i,(int,np.ndarray)):
            return i
        else:
            return i()
    v_vectorized = np.vectorize(v_mapper,otypes=[np.object])
    
    def a_mapper(i,q,qd):
        if len(i)==3:
            fun,bi,bj = i
            return fun(q[bi],q[bj],qd[bi],qd[bj])
        else:
            fun,bi = i
            return fun(qd[bi])
    a_vectorized = np.vectorize(a_mapper,otypes=[np.object],excluded=('q','qd'))
    
    def r_mapper(i,q,lamda):
        fun,bi,bj,index = i
        return fun(q[bi],q[bj],lamda[index].values)
    r_vectorized = np.vectorize(r_mapper,otypes=[np.object],excluded=('q','lamda'))
    
    ####################################################################################
    ####################################################################################
    
    mass_matrix = np.zeros((n_nodes,n_nodes),dtype=np.object)
    mass_matrix.fill(None)
    
    gravitational_vector = np.zeros((n_nodes),dtype=np.object)
    centrifugal_vector   = np.zeros((n_nodes),dtype=np.object)
    inertiaforce_vector  = np.zeros((n_nodes),dtype=np.object)
    
    for n in nodelist:
        ni = node_index[n]
        ni7 = (ni*7)+n7
        
        mass_matrix[ni,ni]=(n.mass_matrix,ni7)
        
        gravitational_vector[ni]=n.gravity
        
        centrifugal_vector[ni]=(n.centrifugal,ni7)
        
        inertiaforce_vector[ni]=(n.inertia_force,ni7)
        
        
    def mass_mapper(i,q):
        fun,bi = i
        return fun(q[bi])
    mass_vectorized = np.vectorize(mass_mapper,otypes=[np.object],excluded='q')
    
    def centrifugal_mapper(i,q,qd):
        fun,bi = i
        return fun(q[bi],qd[bi])
    centrifugal_vectorized = np.vectorize(centrifugal_mapper,otypes=[np.object],excluded=('q','qd'))
    
    def gravity_mapper(i):
        return i()
    gravity_vectorized = np.vectorize(gravity_mapper,otypes=[np.object])
    
    
    system = pd.Series([jacobian,
                        jacobian.nonzero(),
                        equations, 
                        vel_rhs, 
                        acc_rhs,
                        reaction_vector,
                        reaction_indecies,
                        lamda_vector,
                        mass_matrix,
                        gravitational_vector,
                        centrifugal_vector,
                        inertiaforce_vector,
                        j_vectorized,
                        v_vectorized,
                        a_vectorized,
                        r_vectorized,
                        mass_vectorized,
                        centrifugal_vectorized,
                        gravity_vectorized],
                       index=['Jacobian','Jnzi','Ceq','Veq','Aeq','Rvec','Rind','Lind',
                              'MassMatrix','Gravity','Centrifugal','Inertia',
                              'Jf','Vf','Af','Rf','Mf','Cf','Gf'])
    
    return system


In [16]:
def jacobian_evaluator(jac_blocks,nzi,mapper,q,Id=None):
    A = jac_blocks.copy()
    A[nzi]=mapper(A[nzi],q=q)
    
    if Id is None:
        return sc.sparse.bmat(A,format='csc')
    else:
        J = sc.sparse.bmat(A,format='csc')
        return sc.sparse.bmat([[J],[Id]],format='csc')
    

In [17]:
def nr_kds(jac_blocks,equations_blocks,nzi,j_vectorized,guess,Id=None):
    
    A = jacobian_evaluator(jac_blocks,nzi,j_vectorized,guess,Id)
    
    b = equations_blocks.copy()
    bn = np.concatenate(j_vectorized(b[b.nonzero()],q=guess))
    
    if Id is not None:
        nqind=Id.shape[0]
        Ieq=np.zeros((nqind,1))
        bn = np.concatenate([bn,Ieq])
        
    delta_q = sc.sparse.linalg.spsolve(A,-bn)
    
    itr=0
    while np.linalg.norm(delta_q)>1e-5:
        
        guess=guess+delta_q
        
        if itr!=0 and itr%5==0:
            print('Recalculating Jacobian')
            A = jacobian_evaluator(jac_blocks,nzi,j_vectorized,guess,Id)
        bn = np.concatenate(j_vectorized(b[b.nonzero()],q=guess))
        if Id is not None:
            nqind=Id.shape[0]
            Ieq=np.zeros((nqind,1))
            bn = np.concatenate([bn,Ieq])
        delta_q = sc.sparse.linalg.spsolve(A,-bn)
        
        itr+=1
        
        if itr>200:
            print("Iterations exceded \n")
            break    
    
    return guess,jacobian_evaluator(jac_blocks,nzi,j_vectorized,guess,Id)


In [18]:
import scipy as sc

def coordinates_mapper(q):
    n=len(q)
    int2str=pd.Series(q.index,index=np.arange(0,n,1))
    str2int=pd.Series(np.arange(0,n,1),index=q.index)
    return int2str, str2int

def extract_ind(sparse_jac,q):
    mat=sparse_jac.A
    rows,cols=mat.shape
    permR=sc.linalg.lu(mat.T)[0]
    ind_cols=permR[:,rows:]
    maped=coordinates_mapper(q)[0]
    ind_coord=[maped[np.argmax(ind_cols[:,i])] for i in range(cols-rows) ]
    
    return ind_coord, ind_cols, permR

def assign_initial_conditions(q0,qd0,qind):
    q_initial  = list(q0[qind])
    qd_initial = list(qd0[qind])
    return q_initial+qd_initial



def dds(q0,qd0,system_equations,time_array):
    '''
    Dynamically Driven Systems Solver
    '''
    #Qa_f = system_equations.Qa
    Cf = system_equations.Cf
    Gf = system_equations.Gf
    Mf  = system_equations.Mf
    Jf = eq_f = system_equations.Jf
    Af = system_equations.Af
    Vf = system_equations.Vf
    Rf = system_equations.Rf
    
    Cv = system_equations.Centrifugal
    Gv = system_equations.Gravity
    Mm  = system_equations.MassMatrix
    Jm = eq_f = system_equations.Jacobian
    Av = system_equations.Aeq
    Vv = system_equations.Veq
    Ev = system_equations.Ceq
    Rv = system_equations.Rvec
    Jnzi = system_equations.Jnzi
    
    # creating dataframes to hold the simulation results at each timestep
    # with initial conditions at t0
    position_df=pd.DataFrame(columns=q0.index)
    velocity_df=pd.DataFrame(columns=q0.index)
    acceleration_df=pd.DataFrame(columns=q0.index)
    lamda_df=pd.DataFrame(columns=np.concatenate(system_equations.Lind))
    JR_df=pd.DataFrame(columns=np.concatenate(system_equations.Rind))
    
    position_df.loc[0]=q0
    velocity_df.loc[0]=qd0
    
    # assigning initial conditions to the system NE equations
    M  = sc.sparse.block_diag(Mf(Mm.diagonal(),q=q0.values),format='csc')
    J  = jacobian_evaluator(Jm,Jnzi,Jf,q0.values)
    
    # Initiating coordinate partitioning
    qind=extract_ind(J,q0)
    qstr=qind[0]
    Ids=qind[1]
    perm_matrix = sc.sparse.csc_matrix(qind[2]).T
    qind_index=list(coordinates_mapper(q0)[1][qstr])
    dof = len(qstr)
    print('DOF : %s \n' %dof)
    print('Independent Coordinates are: %s with indices: %s \n'%(qstr,qind_index))
    
    
    Qv = np.concatenate(Cf(Cv,q=q0.values,qd=qd0.values))
    Qg = np.concatenate(Gf(Gv))
    Qt = (Qv+Qg)
    Qd = np.concatenate(Af(Av,q=q0.values,qd=qd0.values))
       
    
    # creating dataframes to hold the simulation results at each timestep
    # with initial conditions at t0
    position_df=pd.DataFrame(columns=q0.index)
    velocity_df=pd.DataFrame(columns=q0.index)
    acceleration_df=pd.DataFrame(columns=q0.index)
    lamda_df=pd.DataFrame(columns=np.concatenate(system_equations.Lind))
    JR_df=pd.DataFrame(columns=np.concatenate(system_equations.Rind))
    
    position_df.loc[0]=q0
    velocity_df.loc[0]=qd0
    
    
    # assembling the coefficient matrix and the rhs vector and solving for
    # system accelerations and lagrange multipliers
    coeff_matrix=sc.sparse.bmat([[M,J.T],[J,None]],format='csc')
    b_vector=np.concatenate([Qt,-Qd])
    x=sc.sparse.linalg.spsolve(coeff_matrix,b_vector)
    
    # updating the dataframes with the evaluated results
    qdd0n  = x[:len(q0)] # the first 7xnb elements in the x vector
    lamda0 = x[len(q0):] # the rest of elements in the x vector
    acceleration_df.loc[0]=qdd0n
    lamda_df.loc[0]=lamda0
    
    reaction=Rf(Rv,q=q0.values,lamda=lamda_df.loc[0])
    JR_df.loc[0]=np.concatenate(reaction)

    
    # Setting up the integrator function and the initial conditions
    
    def ssm(t,y,Mii,Mid,Qti,Cqi,lamda,qdd_d):
        
        v=list(y[len(y)//2:])
        vdot=sc.sparse.linalg.spsolve(Mii,(Qti.flatten()-(Cqi.T@lamda)-(Mid@qdd_d)))
        vdot=list(vdot)
        dydt=v+vdot
        return dydt
    

    Mp  = perm_matrix@M@perm_matrix.T
    Mii = Mp[-dof:,-dof:]
    Mid = Mp[-dof:,:-dof]
    
    Qtp = perm_matrix@Qt
    Qti = Qtp[-dof:]
    
    Cqp = J@perm_matrix.T
    Cqi = Cqp[:,-dof:]
    
    qdd_d = (perm_matrix@qdd0n)[:-dof]
    
    
    #return position_df,velocity_df,acceleration_df,JR_df, acc_k
    
    r=sc.integrate.ode(ssm).set_integrator('dop853')
    y0=assign_initial_conditions(q0,velocity_df.loc[0],qstr)
    r.set_initial_value(y0).set_f_params(Mii,Mid,Qti,Cqi,lamda0,qdd_d)
    
    # Setting up the time array to be used in integration steps and starting
    # the integration
    for i,dt in enumerate(time_array):
#        print('time_step: '+str(i))
        #progress_bar(len(t),i)

        r.integrate(dt)
        print(r.y)
#        if not r.successful():
#            print("BREAKING SOLVER")
#            return position_df,velocity_df,acceleration_df,JR_df
        
        # creating the guess vector for the vector q as the values of the 
        # previous step and the value of newly evaluated independent coordinate
        guess=position_df.loc[i]+velocity_df.loc[i]*dt + 0.5*acceleration_df.loc[i]*dt**2
        guess[qstr]=r.y[:len(r.y)//2]
        
        # Evaluating the dependent vector q using newton raphson
        dependent=nr_kds(Jm,Ev,Jnzi,Jf,guess,Ids.T)
        position_df.loc[i+1]=dependent[0]
        Cq_new=dependent[1]
        # Calculating the system velocities
        vrhs=np.concatenate(Vf(Vv))
        vind=np.array(r.y[len(r.y)//2:]).reshape((len(r.y)//2,1))
        vrhs=np.concatenate([-vrhs,vind])
        velocity_df.loc[i+1]=sc.sparse.linalg.spsolve(Cq_new,vrhs)
        
        
        q=position_df.loc[i+1]
        qd=velocity_df.loc[i+1]
        
        # Evaluating the new coeff matrix bloks of the system NE equations
        M  = sc.sparse.block_diag(Mf(Mm.diagonal(),q=q.values),format='csc')
        J  = jacobian_evaluator(Jm,Jnzi,Jf,q.values)
        Qv = np.concatenate(Cf(Cv,q=q.values,qd=qd.values))
        Qg = np.concatenate(Gf(Gv))
        Qt = (Qv+Qg)
        Qd = np.concatenate(Af(Av,q=q.values,qd=qd.values))

        coeff_matrix=sc.sparse.bmat([[M,J.T],[J,None]],format='csc')
        b_vector=np.concatenate([Qt,-Qd])
        # Evaluating the acceleration and lagrange mult. vector
        x=sc.sparse.linalg.spsolve(coeff_matrix,b_vector)
        
        # updating the dataframes with the evaluated results
        qdd   = x[:len(q0)] # the first 7xnb elements in the x vector
        lamda = x[len(q0):] # the rest of elements in the x vector
        acceleration_df.loc[i+1]=qdd
        lamda_df.loc[i+1]=lamda
        reaction=Rf(Rv,q=q.values,lamda=lamda_df.loc[i+1])
        JR_df.loc[i+1]=np.concatenate(reaction)
        

        # Setting the ssm input parameters
        Mp = perm_matrix@M@perm_matrix.T
        Mii = Mp[-dof:,-dof:]
        Mid = Mp[-dof:,:-dof]

        Qtp = perm_matrix@Qt
        Qti = Qtp[-dof:]

        Cqp = J@perm_matrix.T
        Cqi = Cqp[:,-dof:]

        qdd_d = (perm_matrix@qdd0n)[:-dof]
        
        r.set_f_params(Mii,Mid,Qti,Cqi,lamda,qdd_d)
    
    
    return position_df,velocity_df,acceleration_df,JR_df


In [69]:
def principle_inertia(J):
    '''
    extracting the principle axes and the corresponding principle moment of
    inertia values from the inertia tensor calculated at the body cm aligned
    with the global frame.
    The process is done by evaluating the eigen values and eigen vectors of the
    J matrix.
    ===========================================================================
    inputs  : 
        J   : Inertia tensor
    ===========================================================================
    outputs : 
        C   : 3x3 ndarray representing the orientation of the body with the
              three principle axes
        Jp  : 3x3 diagonal ndarray storing the principle values in the diagonal
    ===========================================================================
    '''
    PJ,C=np.linalg.eig(J)
    J_Principle=np.diag(PJ)
    #C[:,2] = vec2skew(vector(C[:,0]))@C[:,1]
    # C matrix transform from the body frame to the global frame
    return C, J_Principle

In [70]:
principle_inertia(m.bodies['rbs_link'].J)

(array([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]]),
 array([[  6.45267214e+09,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   6.45267214e+09,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   3.87044215e+06]]))

In [100]:
a = np.array([[0,-1,0],
              [0, 0,1],
              [1,0, 0]])
dcm2ep(a)


array([ 0.5, -0.5, -0.5, -0.5])

In [19]:
q0=pd.concat([i.q0 for i in m.topology.nodes])
qd0 = 0*q0
assembled = equations_assembler(m.topology)

In [101]:
q0=pd.concat([i.q0 for i in m.topology.nodes])
q0['rbs_link.e0':'rbs_link.e3']=[ 0.5, -0.5, -0.5, -0.5]
qd0 = 0*q0
assembled = equations_assembler(m.topology)

Cf = assembled.Cf
Gf = assembled.Gf
Mf  = assembled.Mf
Jf = eq_f = assembled.Jf
Af = assembled.Af
Vf = assembled.Vf
Rf = assembled.Rf

Cv = assembled.Centrifugal
Gv = assembled.Gravity
Mm = assembled.MassMatrix
Jm = assembled.Jacobian
Av = assembled.Aeq
Vv = assembled.Veq
Ev = assembled.Ceq
Rv = assembled.Rvec
Jnzi = assembled.Jnzi

# creating dataframes to hold the simulation results at each timestep
position_df=pd.DataFrame(columns=q0.index)
velocity_df=pd.DataFrame(columns=q0.index)
acceleration_df=pd.DataFrame(columns=q0.index)
lamda_df=pd.DataFrame(columns=np.concatenate(assembled.Lind))
JR_df=pd.DataFrame(columns=np.concatenate(assembled.Rind))

position_df.loc[0]=q0
velocity_df.loc[0]=qd0

# assigning initial conditions to the system NE equations
M  = sc.sparse.block_diag(Mf(Mm.diagonal(),q=q0.values),format='csc')
J  = jacobian_evaluator(Jm,Jnzi,Jf,q0.values)

# Initiating coordinate partitioning
qind=extract_ind(J,q0)
qstr=qind[0]
Ids=qind[1]
perm_matrix = sc.sparse.csc_matrix(qind[2]).T
qind_index=list(coordinates_mapper(q0)[1][qstr])
dof = len(qstr)
print('DOF : %s \n' %dof)
print('Independent Coordinates are: %s with indices: %s \n'%(qstr,qind_index))


Qv = np.concatenate(Cf(Cv,q=q0.values,qd=qd0.values))
Qg = np.concatenate(Gf(Gv))
Qg[2]=0
Qt = (Qv+Qg)
Qd = np.concatenate(Af(Av,q=q0.values,qd=qd0.values))


# assembling the coefficient matrix and the rhs vector and solving for
# system accelerations and lagrange multipliers
coeff_matrix=sc.sparse.bmat([[M,J.T],[J,None]],format='csc')
b_vector=np.concatenate([Qt,-Qd])
x=sc.sparse.linalg.spsolve(coeff_matrix,b_vector)

# updating the dataframes with the evaluated results
qdd0n  = x[:len(q0)] # the first 7xnb elements in the x vector
lamda0 = x[len(q0):] # the rest of elements in the x vector
acceleration_df.loc[0]=qdd0n
lamda_df.loc[0]=lamda0
Qc = -J.T@lamda0

reaction=Rf(Rv,q=q0.values,lamda=lamda_df.loc[0])
JR_df.loc[0]=np.concatenate(reaction)

Mp  = perm_matrix@M@perm_matrix.T
Mii = Mp[-dof:,-dof:]
Mid = Mp[-dof:,:-dof]

Qtp = perm_matrix@Qt
Qti = Qtp[-dof:]

Cqp = J@perm_matrix.T
Cqi = Cqp[:,-dof:]

qdd_d = (perm_matrix@qdd0n)[:-dof]

DOF : 1 

Independent Coordinates are: ['rbs_link.z'] with indices: [9] 



In [111]:
Mp  = perm_matrix@M@perm_matrix.T
Mii = Mp[-dof:,-dof:]
Mid = Mp[-dof:,:-dof]
Mdd = Mp[:-dof,:-dof]
Mdi = Mp[:-dof,-dof:]

Qtp = perm_matrix@Qt
Qti = Qtp[-dof:]

Cqp = J@perm_matrix.T
Cqd = Cqp[:,:-dof]
Cqi = Cqp[:,-dof:]

#H = -1*sc.sparse.linalg.inv(Cqd)@Cqi

qdd_d = (perm_matrix@qdd0n)[:-dof]

#Mcab = Mii+(Mid@H)+H.T@(Mdi+Mdd@H)
pd.DataFrame(Cqd.A)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-1000.0,1000.0,-1000.0,-1000.0,0.000000e+00,1.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,-1.0
1,-1000.0,-1000.0,1000.0,-1000.0,0.000000e+00,0.0,1.0,0.0,-1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0
2,1000.0,1000.0,1000.0,-1000.0,0.000000e+00,0.0,0.0,1.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0
3,1.0,-1.0,-1.0,-1.0,4.474229e-17,0.0,0.0,0.0,0.0,7.850462e-17,2.000000e+00,7.850462e-17,0.0
4,-1.0,-1.0,1.0,-1.0,2.000000e+00,0.0,0.0,0.0,0.0,7.850462e-17,-4.474229e-17,-7.850462e-17,0.0
5,0.0,0.0,0.0,0.0,0.000000e+00,1.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0
6,0.0,0.0,0.0,0.0,0.000000e+00,0.0,1.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0
7,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0
8,0.0,0.0,0.0,0.0,1.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0
9,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,1.000000e+00,0.000000e+00,0.000000e+00,0.0


In [102]:
q0

rbs_ground.x        0.0
rbs_ground.y        0.0
rbs_ground.z        0.0
rbs_ground.e0       1.0
rbs_ground.e1       0.0
rbs_ground.e2       0.0
rbs_ground.e3       0.0
rbs_link.x       1000.0
rbs_link.y          0.0
rbs_link.z          0.0
rbs_link.e0         0.5
rbs_link.e1        -0.5
rbs_link.e2        -0.5
rbs_link.e3        -0.5
dtype: float64

In [103]:
lamda_df

,jcs_rev_eq0,jcs_rev_eq1,jcs_rev_eq2,jcs_rev_eq3,jcs_rev_eq4,rbs_ground_eq0,rbs_ground_eq1,rbs_ground_eq2,rbs_ground_eq3,rbs_ground_eq4,rbs_ground_eq5,rbs_ground_eq6,rbs_link_eq0
0,2.565680e-09,0.000003,4.747197e+07,0.000002,1.258061e+11,-2.567044e-09,-0.000003,-4.747197e+07,-2.516122e+11,-0.00001,9.958713e-07,0.00001,-1.258061e+11


In [104]:
Qc

array([  1.36424205e-12,   1.28714303e-09,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   2.56567962e-09,   3.10615506e-06,
         4.74719748e+07,  -4.74719748e+10,  -4.74719748e+10,
        -4.74719748e+10,   4.74719748e+10])

In [105]:
JR_df

,jcs_rev_Fx,jcs_rev_Fy,jcs_rev_Fz,jcs_rev_Mx,jcs_rev_My,jcs_rev_Mz
0,-2.565680e-09,-0.000003,-4.747197e+07,-0.000005,4.979357e-07,0.000005


In [106]:
acceleration_df

,rbs_ground.x,rbs_ground.y,rbs_ground.z,rbs_ground.e0,rbs_ground.e1,rbs_ground.e2,rbs_ground.e3,rbs_link.x,rbs_link.y,rbs_link.z,rbs_link.e0,rbs_link.e1,rbs_link.e2,rbs_link.e3
0,1.364242e-12,1.287143e-09,1.818989e-12,-0.0,-0.0,-0.0,-0.0,1.325781e-13,1.605065e-10,-7356.948229,-1.839237,-1.839237,-1.839237,1.839237


In [77]:
((-1.06787918e+08+0*1.072625e+08)/10885.618545)*1e-3

-9.810000006756622

In [29]:
pd.DataFrame(M.A)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19352.210746,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,19352.210746,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,19352.210746,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [30]:
pd.DataFrame(J.A)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,-1.0,0.0,0.0,-5.411961e+02,1.306563e+03,-5.411961e+02,1.306563e+03
1,0.0,1.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,-1.0,0.0,-1.306563e+03,5.411961e+02,1.306563e+03,-5.411961e+02
2,0.0,0.0,1.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,-1.0,-5.411961e+02,-1.306563e+03,5.411961e+02,1.306563e+03
3,0.0,0.0,0.0,1.244916e-16,1.414214,1.895269e-16,1.414214,0.0,0.0,0.0,1.847759e+00,-1.028123e-16,-3.443941e-16,7.653669e-01
4,0.0,0.0,0.0,1.895269e-16,1.414214,-1.244916e-16,-1.414214,0.0,0.0,0.0,1.456821e-16,7.653669e-01,1.847759e+00,-1.028123e-16
5,1.0,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,0.0,1.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,0.0,0.0,1.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,0.0,0.0,0.0,1.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,0.0,0.0,0.0,0.000000e+00,1.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


time_array = np.arange(5e-4,0.5,5e-4)
d= dds(q0,0*q0,assembled,time_array)

In [88]:
d[2]

,rbs_ground.x,rbs_ground.y,rbs_ground.z,rbs_ground.e0,rbs_ground.e1,rbs_ground.e2,rbs_ground.e3,rbs_link.x,rbs_link.y,rbs_link.z,rbs_link.e0,rbs_link.e1,rbs_link.e2,rbs_link.e3
0,1.264198e-10,4.231733e-13,-1.364242e-10,-0.0,-0.0,-0.0,-0.0,-3678.439632,1.957997e-13,-3678.439632,1.029404,1.341546,-0.426393,-3.238779


In [42]:
assembled

Jacobian       [[(<bound method revolute.jacobian_i of <const...
Jnzi                                ([0, 0, 1, 2], [0, 1, 0, 1])
Ceq            [(<bound method revolute.equations of <constra...
Veq            [[[0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], ...
Aeq            [(<bound method revolute.acc_rhs of <constrain...
Rvec           [(<bound method joint.reactions of <constraint...
Rind           [[jcs_rev_Fx, jcs_rev_Fy, jcs_rev_Fz, jcs_rev_...
Lind           [[jcs_rev_eq0, jcs_rev_eq1, jcs_rev_eq2, jcs_r...
MassMatrix     [[(<bound method rigid.mass_matrix of rbs_grou...
Gravity        [<bound method rigid.gravity of rbs_ground>, <...
Centrifugal    [(<bound method rigid.centrifugal of rbs_groun...
Inertia        [(<bound method rigid.inertia_force of rbs_gro...
Jf             <numpy.lib.function_base.vectorize object at 0...
Vf             <numpy.lib.function_base.vectorize object at 0...
Af             <numpy.lib.function_base.vectorize object at 0...
Rf             <numpy.lib

In [45]:
assembled['Aeq']

array([ (<bound method revolute.acc_rhs of <constraints.revolute object at 0x00000176522FDD68>>, array([0, 1, 2, 3, 4, 5, 6]), array([ 7,  8,  9, 10, 11, 12, 13])),
       (<bound method mount.acc_rhs of rbs_ground>, array([0, 1, 2, 3, 4, 5, 6])),
       (<bound method rigid.acc_rhs of rbs_link>, array([ 7,  8,  9, 10, 11, 12, 13]))], dtype=object)

In [29]:
d[3]

,jcs_rev_Fx,jcs_rev_Fy,jcs_rev_Fz,jcs_rev_Mx,jcs_rev_My,jcs_rev_Mz
0,-1.127839e-08,2.017354e-09,-2.670036e+07,0.000002,5.091324e-22,0.000004


In [115]:
d[4]

-0.0

In [11]:
def plots(i):
    plt.figure(figsize=(10,20))

    plt.subplot(611)
    plt.title('z')
    plt.plot(time_array[:i],d[0]['rbs_link.z'][:i])

    plt.subplot(612)
    plt.title('x')
    plt.plot(time_array[:i],d[0]['rbs_link.x'][:i])
    
    plt.subplot(613)
    plt.title('e0')
    plt.plot(time_array[:i],d[0]['rbs_link.e0'][:i])
    
    plt.subplot(614)
    plt.title('e1')
    plt.plot(time_array[:i],d[0]['rbs_link.e1'][:i])
    
    plt.subplot(615)
    plt.title('e2')
    plt.plot(time_array[:i],d[0]['rbs_link.e2'][:i])
    
    plt.subplot(616)
    plt.title('e3')
    plt.plot(time_array[:i],d[0]['rbs_link.e3'][:i])

    plt.grid()
    plt.show()

widgets.interact(plots,i=widgets.IntSlider(value=1,max=199))

interactive(children=(IntSlider(value=1, description='i', max=199), Output()), _dom_classes=('widget-interact'…

<function __main__.plots>

In [49]:
def plots(i):
    plt.figure(figsize=(10,20))

    plt.subplot(611)
    plt.title('z')
    plt.plot(time_array[:i],d[1]['rbs_link.z'][:i])

    plt.subplot(612)
    plt.title('x')
    plt.plot(time_array[:i],d[1]['rbs_link.x'][:i])
    
    plt.subplot(613)
    plt.title('e0')
    plt.plot(time_array[:i],d[1]['rbs_link.e0'][:i])
    
    plt.subplot(614)
    plt.title('e1')
    plt.plot(time_array[:i],d[1]['rbs_link.e1'][:i])
    
    plt.subplot(615)
    plt.title('e2')
    plt.plot(time_array[:i],d[1]['rbs_link.e2'][:i])
    
    plt.subplot(616)
    plt.title('e3')
    plt.plot(time_array[:i],d[1]['rbs_link.e3'][:i])

    plt.grid()
    plt.show()

widgets.interact(plots,i=widgets.IntSlider(value=1,max=200))

interactive(children=(IntSlider(value=1, description='i', max=200), Output()), _dom_classes=('widget-interact'…

<function __main__.plots>

In [57]:

def plots(i):
    plt.figure(figsize=(10,20))

    plt.subplot(611)
    plt.title('z')
    plt.plot(time_array[:i],d[2]['rbs_link.z'][:i])

    plt.subplot(612)
    plt.title('x')
    plt.plot(time_array[:i],d[2]['rbs_link.x'][:i])
    
    plt.subplot(613)
    plt.title('e0')
    plt.plot(time_array[:i],d[2]['rbs_link.e0'][:i])
    
    plt.subplot(614)
    plt.title('e1')
    plt.plot(time_array[:i],d[2]['rbs_link.e1'][:i])
    
    plt.subplot(615)
    plt.title('e2')
    plt.plot(time_array[:i],d[2]['rbs_link.e2'][:i])
    
    plt.subplot(616)
    plt.title('e3')
    plt.plot(time_array[:i],d[2]['rbs_link.e3'][:i])

    plt.grid()
    plt.show()

widgets.interact(plots,i=widgets.IntSlider(value=1,max=200))

interactive(children=(IntSlider(value=1, description='i', max=200), Output()), _dom_classes=('widget-interact'…

<function __main__.plots>